# TITANIC

In [ ]:
#dftrainr = raw
#dftrainm = modificado tipos de variables


## 1. Install and Import

In [ ]:
! pip install ydata-profiling

In [ ]:
! pip install matplotlib

In [ ]:
!pip install pycaret

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [30]:
dftrainr = pd.read_csv(r'train.csv')
dftestr = pd.read_csv(r'test.csv')

## 2. EDA

In [ ]:
dftrainr.shape


In [ ]:
dftrainr.describe()

In [ ]:
dftrainr.info()

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(dftrainr)
profile.to_file("ydata-profiling EDA Titanic_X.html")

In [ ]:
dftrainr_boxplot = dftrainr.dropna(subset=['Age', 'Survived'])

plt.figure(figsize=(10, 6))
sns.boxplot(x='Survived', y='Age', data=dftrainr_boxplot)

plt.title('Boxplots de Edades por Survived')
plt.show()

## 3. Data Processing

In [115]:
dftrainl = dftrainr.copy()

### Imputar (o eliminar) faltantes de Embarked

In [116]:
dftrainl.loc[dftrainl['PassengerId'] == 62, 'Embarked'] = 'S'
dftrainl.loc[dftrainl['PassengerId'] == 830, 'Embarked'] = 'S'

### Eliminar Columnas

In [117]:
dftrainl = dftrainl.drop(['Cabin', 'Name', 'Ticket'],  axis=1)

### Cambiar tipo: Embarked (a dummy)

In [118]:
# dftrainr['Embarked'] = dftrainr['Embarked'].astype('category') 

In [119]:
dftrainl = pd.get_dummies(dftrainl, columns=['Embarked'], prefix='Embarked')

### Cambiar tipo: Pclass a dummy

In [120]:
# Pclass a Categórica (la codificación ordinal ya está inherente en 1,2,3)
# dftrainr['Pclass'] = dftrainr['Pclass'].astype('category')
dftrainl = pd.get_dummies(dftrainl, columns=['Pclass'], prefix='Pclass')

### Cambiar tipo: Sex a dummy (one-hot)

In [121]:
# dftrainr['Sex'] = dftrainr['Sex'].map({'male': 0, 'female': 1})
dftrainl = pd.get_dummies(dftrainl, columns=['Sex'], prefix='Sex')

### Cambiar tipo: para modelo_survived uso  bool / para modelo_age uso dummy

In [122]:
dftrainm = dftrainl.copy()
dftrainm['Survived'] = dftrainm['Survived'].astype(bool) # 0=False

In [123]:
dftrainl = pd.get_dummies(dftrainl, columns=['Survived'], prefix='Survived')

### Imputar (o eliminar) faltantes de Age

In [124]:
dftrainl[dftrainl['Age'].isnull()].head(5)

,PassengerId,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Survived_0,Survived_1
5,6,NaN,0,0,8.4583,0,1,0,0,0,1,0,1,1,0
17,18,NaN,0,0,13.0000,0,0,1,0,1,0,0,1,0,1
19,20,NaN,0,0,7.2250,1,0,0,0,0,1,1,0,0,1
26,27,NaN,0,0,7.2250,1,0,0,0,0,1,0,1,1,0
28,29,NaN,0,0,7.8792,0,1,0,0,0,1,1,0,0,1


In [125]:
dftrainr[dftrainr['Age'] < 1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
78,79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,248738,29.0000,NaN,S
305,306,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S
469,470,1,3,"Baclini, Miss. Helene Barbara",female,0.75,2,1,2666,19.2583,NaN,C
644,645,1,3,"Baclini, Miss. Eugenie",female,0.75,2,1,2666,19.2583,NaN,C
755,756,1,2,"Hamalainen, Master. Viljo",male,0.67,1,1,250649,14.5000,NaN,S
803,804,1,3,"Thomas, Master. Assad Alexander",male,0.42,0,1,2625,8.5167,NaN,C
831,832,1,2,"Richards, Master. George Sibley",male,0.83,1,1,29106,18.7500,NaN,S


#### Model_Age: Winner

In [126]:
# Crear conjuntos de entrenamiento y prueba
age_train = dftrainl.dropna(subset=['Age'])
age_test = dftrainl[dftrainl['Age'].isnull()]

# Seleccionar características relevantes para el modelo
features = ['SibSp', 'Fare', 'Sex_female', 'Sex_male', "Survived_0", 'Survived_1', 'Pclass_1', 'Pclass_2', 'Pclass_3']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train = age_train[features]
y_train = age_train['Age']
X_test = age_test[features]

# Inicializar y entrenar el modelo predictivo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir edades faltantes
predicted_ages = model.predict(X_test)

# Llenar valores faltantes con predicciones
dftrainm.loc[dftrainm['Age'].isnull(), 'Age'] = predicted_ages

In [127]:
dftrainm['Age'] = dftrainm['Age'].astype('float64')
passenger_ids_to_check = [6, 18, 20, 27, 29, 79]
dftrainm[dftrainm['PassengerId'].isin(passenger_ids_to_check)]
#los valores < 1 quedan con predicción 0

,PassengerId,Survived,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
5,6,False,29.335761,0,0,8.4583,0,1,0,0,0,1,0,1
17,18,True,28.254310,0,0,13.0000,0,0,1,0,1,0,0,1
19,20,True,22.669112,0,0,7.2250,1,0,0,0,0,1,1,0
26,27,False,29.367052,0,0,7.2250,1,0,0,0,0,1,0,1
28,29,True,22.652514,0,0,7.8792,0,1,0,0,0,1,1,0
78,79,True,0.830000,0,2,29.0000,0,0,1,0,1,0,0,1


In [128]:
dftrainm['Age'].isnull().sum()

0

In [175]:
#Guardar DF para DSSW
dftrainm.to_csv('dftrainm.csv', index=False)

#### Performance Test: model_age

In [46]:
# Crear conjuntos de entrenamiento y prueba
age_train = dftrainl.dropna(subset=['Age'])
age_test = dftrainl[dftrainl['Age'].isnull()]

# Seleccionar características relevantes para el modelo
features = ['SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', "Survived_0", 'Survived_1', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train = age_train[features]
y_train = age_train['Age']
X_test = age_test[features]

# Inicializar y entrenar el modelo predictivo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir edades faltantes
predicted_ages = model.predict(X_test)

# Calcular métricas en el conjunto de entrenamiento
mae_train = mean_absolute_error(y_train, model.predict(X_train))
mse_train = mean_squared_error(y_train, model.predict(X_train))
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, model.predict(X_train))

# Imprimir resultados
print(f'MAE en el conjunto de entrenamiento: {mae_train}')
print(f'MSE en el conjunto de entrenamiento: {mse_train}')
print(f'RMSE en el conjunto de entrenamiento: {rmse_train}')
print(f'R² en el conjunto de entrenamiento: {r2_train}')

# Agregar una constante al conjunto de características
X_train = sm.add_constant(X_train)

# Inicializar el modelo de Statsmodels
model_sm = sm.OLS(y_train, X_train)

# Ajustar el modelo
results = model_sm.fit()

# Imprimir los resultados de los p-values
print(results.summary())

MAE en el conjunto de entrenamiento: 9.821087556712884
MSE en el conjunto de entrenamiento: 151.5203897236726
RMSE en el conjunto de entrenamiento: 12.309361873130248
R² en el conjunto de entrenamiento: 0.2809518996364885
                            OLS Regression Results                            
Dep. Variable:                    Age   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     30.56
Date:                Thu, 01 Feb 2024   Prob (F-statistic):           3.29e-45
Time:                        16:25:27   Log-Likelihood:                -2805.5
No. Observations:                 714   AIC:                             5631.
Df Residuals:                     704   BIC:                             5677.
Df Model:                           9                                         
Covariance Type:            nonrobust                              

In [ ]:
# este es mejor

# Crear conjuntos de entrenamiento y prueba
age_train = dftrainl.dropna(subset=['Age'])
age_test = dftrainl[dftrainl['Age'].isnull()]

# Seleccionar características relevantes para el modelo
features = ['SibSp', 'Fare', 'Sex_female', 'Sex_male', "Survived_0", 'Survived_1', 'Pclass_1', 'Pclass_2', 'Pclass_3']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train = age_train[features]
y_train = age_train['Age']
X_test = age_test[features]

# Inicializar y entrenar el modelo predictivo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir edades faltantes
predicted_ages = model.predict(X_test)

# Calcular métricas en el conjunto de entrenamiento
mae_train = mean_absolute_error(y_train, model.predict(X_train))
mse_train = mean_squared_error(y_train, model.predict(X_train))
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, model.predict(X_train))

# Imprimir resultados
print(f'MAE en el conjunto de entrenamiento: {mae_train}')
print(f'MSE en el conjunto de entrenamiento: {mse_train}')
print(f'RMSE en el conjunto de entrenamiento: {rmse_train}')
print(f'R² en el conjunto de entrenamiento: {r2_train}')

# Agregar una constante al conjunto de características
X_train = sm.add_constant(X_train)

# Inicializar el modelo de Statsmodels
model_sm = sm.OLS(y_train, X_train)

# Ajustar el modelo
results = model_sm.fit()

# Imprimir los resultados de los p-values
print(results.summary())

#### Otros clálculos

In [ ]:
# Convertir edades a valores enteros (si lo deseas)


In [ ]:
#Normaliza o estandariza las variables numéricas si es necesario.

In [ ]:
#Título mr, mrs en el nombre
# Extraer títulos de los nombres y convertir 'Name' a categoría
dftrainr['Title'] = dftrainr['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
dftrainr['Name'] = dftrainr['Name'].astype('category')

# Verificar el cambio
print(dftrainr['Name'].dtype)

In [ ]:
#edad <1 año

In [ ]:
#llos que viajaron solos tenían más o menos probabilidad de sobrevivir

In [ ]:
#Clean Overview
dftrainc.describe()

## 4. Model_survived

### Modelo Ganador

In [130]:
y = dftrainm['Survived']
x = dftrainm[['PassengerId', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male']]

#Train-Test Split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

#Reglog
from sklearn.linear_model import LogisticRegression
reglog=LogisticRegression()
reglog.fit(x_train, y_train)

#Predicción del Modelo
predict=reglog.predict(x_test)

#### Model Testing

###### --> P-Value

In [146]:
# Crear Df para p_value, porque da problemas. parece que cuando hago el código para pvalue no puedo poner la que usaré de referencia con las categóricas
#Descubrí que hacer esto no se neceista. Con dftrainm se puede para directamente a sacar Pvalue, solo que debo recordar eliminar 1 de las que que me salen como categóricas 
columnas_dftrainm = dftrainm[['PassengerId', 'Survived', 'Age', 'SibSp', 'Parch', 'Fare']]
columnas_dftrainr = dftrainr[['Pclass', 'Sex', 'Embarked']]
dftrain_pvalue = pd.concat([columnas_dftrainr, columnas_dftrainm], axis=1)


dftrain_pvalue['Embarked'] = dftrain_pvalue['Embarked'].astype('category')
dftrain_pvalue.loc[dftrain_pvalue['PassengerId'] == 62, 'Embarked'] = 'S'
dftrain_pvalue.loc[dftrain_pvalue['PassengerId'] == 830, 'Embarked'] = 'S'

# Eliminar una categoría de 'Embarked'
dftrain_pvalue = pd.get_dummies(dftrain_pvalue, columns=['Embarked'], drop_first=True)
dftrain_pvalue = pd.get_dummies(dftrain_pvalue, columns=['Pclass'], drop_first=True)
dftrain_pvalue = pd.get_dummies(dftrain_pvalue, columns=['Sex'], drop_first=True)


# P-Value
import statsmodels.api as sm

# Definir variables dependientes e independientes
y = dftrain_pvalue['Survived']
X = dftrain_pvalue[['Embarked_Q', 'Embarked_S', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_2', 'Pclass_3', 'Sex_male']]

# Agregar una constante a las variables independientes
X_with_constant = sm.add_constant(X)

# Inicializar el modelo de regresión logística
logit_model = sm.Logit(y, X_with_constant)

# Ajustar el modelo
results = logit_model.fit()

# Obtener los p-values
p_values = results.pvalues

# Mostrar los resultados
print('\nP-values:')
print(p_values)

Optimization terminated successfully.
         Current function value: 0.429810
         Iterations 6

P-values:
const         1.206999e-20
Embarked_Q    7.884465e-01
Embarked_S    1.110650e-01
Age           2.757407e-10
SibSp         5.051468e-05
Parch         4.834143e-01
Fare          5.255537e-01
Pclass_2      8.184646e-05
Pclass_3      1.261083e-15
Sex_male      2.900265e-40
dtype: float64


#### Model #1 - Reglog: All Variables

In [207]:
dftrainm.isnull().sum()

PassengerId    0
Survived       0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
Pclass_1       0
Pclass_2       0
Pclass_3       0
Sex_female     0
Sex_male       0
dtype: int64

In [208]:
dftrainm[0:11]

,PassengerId,Survived,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,1,False,22.000000,1,0,7.2500,0,0,1,0,0,1,0,1
1,2,True,38.000000,1,0,71.2833,1,0,0,1,0,0,1,0
2,3,True,26.000000,0,0,7.9250,0,0,1,0,0,1,1,0
3,4,True,35.000000,1,0,53.1000,0,0,1,1,0,0,1,0
4,5,False,35.000000,0,0,8.0500,0,0,1,0,0,1,0,1
5,6,False,29.335761,0,0,8.4583,0,1,0,0,0,1,0,1
6,7,False,54.000000,0,0,51.8625,0,0,1,1,0,0,0,1
7,8,False,2.000000,3,1,21.0750,0,0,1,0,0,1,0,1
8,9,True,27.000000,0,2,11.1333,0,0,1,0,0,1,1,0
9,10,True,14.000000,1,0,30.0708,1,0,0,0,1,0,1,0


In [210]:
dftrainm.shape

(891, 14)

In [182]:

891*0.33
#295

294.03000000000003

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [328]:
# Define variables
y_target = dftrainm['Survived']
X_features = dftrainm[['PassengerId', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male']]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=.3333, random_state=42)

# Logistic Regression Model
reglog = LogisticRegression()
reglog.fit(X_train, y_train)

# Prediction
predict = reglog.predict(X_test)

# Define Metrics
accuracy = accuracy_score(y_test, predict)
precision = precision_score(y_test, predict)
recall = recall_score(y_test, predict)
f1 = f1_score(y_test, predict)
conf_matrix = confusion_matrix(y_test, predict)
classification_rep = classification_report(y_test, predict)

# Print Metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix and Classification Report
print('\nConfusion Matrix:')
print(pd.DataFrame(conf_matrix, columns=['Predicted No', 'Predicted Yes'], index=['Actual No', 'Actual Yes']))
print('\nClassification Report:')
print(classification_rep)

Accuracy: 0.7912
Precision: 0.7788
Recall: 0.6750
F1 Score: 0.7232

Confusion Matrix:
            Predicted No  Predicted Yes
Actual No            154             23
Actual Yes            39             81

Classification Report:
              precision    recall  f1-score   support

       False       0.80      0.87      0.83       177
        True       0.78      0.68      0.72       120

    accuracy                           0.79       297
   macro avg       0.79      0.77      0.78       297
weighted avg       0.79      0.79      0.79       297



#### Model #2 - Reglog: Variables considerando p-value significativo

In [192]:
# Define variables
y_target = dftrainm['Survived']
X_features = dftrainm[['PassengerId', 'SibSp','Embarked_C', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male']]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3333, random_state=42)

# Logistic Regression Model
reglog = LogisticRegression()
reglog.fit(X_train, y_train)

# Prediction
predict = reglog.predict(X_test)

# Define Metrics
accuracy = accuracy_score(y_test, predict)
precision = precision_score(y_test, predict)
recall = recall_score(y_test, predict)
f1 = f1_score(y_test, predict)
conf_matrix = confusion_matrix(y_test, predict)
classification_rep = classification_report(y_test, predict)

# Print Metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Confusion Matrix and Classification Report
print('\nConfusion Matrix:')
print(pd.DataFrame(conf_matrix, columns=['Predicted No', 'Predicted Yes'], index=['Actual No', 'Actual Yes']))
print('\nClassification Report:')
print(classification_rep)

Accuracy: 0.7879
Precision: 0.7478
Recall: 0.7167
F1 Score: 0.7319

Confusion Matrix:
            Predicted No  Predicted Yes
Actual No            148             29
Actual Yes            34             86

Classification Report:
              precision    recall  f1-score   support

       False       0.81      0.84      0.82       177
        True       0.75      0.72      0.73       120

    accuracy                           0.79       297
   macro avg       0.78      0.78      0.78       297
weighted avg       0.79      0.79      0.79       297



#### Model #3 - pycaret: model testing

In [335]:
from pycaret.classification import *

In [336]:
setup = setup(data=dftrainm,
              target="Survived",
              numeric_features=['SibSp', 'Parch', 'Fare'],
              categorical_features=['Embarked_C', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male'],
              ignore_features=['PassengerId'])

,Description,Value
0,Session id,4434
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 14)"
4,Transformed data shape,"(891, 13)"
5,Transformed train set shape,"(623, 13)"
6,Transformed test set shape,"(268, 13)"
7,Ignore features,1
8,Ordinal features,8
9,Numeric features,3


In [337]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [338]:
best = compare_models(sort='Accuracy')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8362,0.8844,0.7281,0.8311,0.7729,0.6459,0.6524,0.0230
lightgbm,Light Gradient Boosting Machine,0.8298,0.8787,0.7321,0.8129,0.7668,0.6337,0.6395,0.0440
rf,Random Forest Classifier,0.8249,0.8709,0.7406,0.7978,0.7649,0.6260,0.6304,0.0370
lr,Logistic Regression,0.8137,0.8608,0.7071,0.7939,0.7414,0.5977,0.6057,0.0450
ridge,Ridge Classifier,0.8121,0.0000,0.6987,0.7923,0.7384,0.5935,0.5998,0.0130
lda,Linear Discriminant Analysis,0.8105,0.8618,0.6987,0.7885,0.7368,0.5904,0.5963,0.0120
ada,Ada Boost Classifier,0.8089,0.8664,0.7154,0.7747,0.7402,0.5899,0.5944,0.0230
et,Extra Trees Classifier,0.7976,0.8423,0.7156,0.7507,0.7286,0.5680,0.5728,0.0370
nb,Naive Bayes,0.7816,0.8348,0.7236,0.7152,0.7165,0.5394,0.5420,0.0130
dt,Decision Tree Classifier,0.7574,0.7466,0.6777,0.6860,0.6803,0.4851,0.4867,0.0130


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [339]:
modelo = create_model("lr")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8730,0.9338,0.9167,0.7857,0.8462,0.7391,0.7455
1,0.7460,0.8152,0.6667,0.6667,0.6667,0.4615,0.4615
2,0.8571,0.8536,0.7500,0.8571,0.8000,0.6897,0.6934
3,0.7419,0.8498,0.4583,0.7857,0.5789,0.4109,0.4420
4,0.8548,0.8481,0.7917,0.8261,0.8085,0.6917,0.6921
5,0.7903,0.7654,0.6667,0.7619,0.7111,0.5477,0.5507
6,0.8548,0.9057,0.7083,0.8947,0.7907,0.6819,0.6928
7,0.7742,0.8640,0.7500,0.6923,0.7200,0.5313,0.5325
8,0.8548,0.9534,0.6667,0.9412,0.7805,0.6767,0.6992


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [327]:
evaluate_model(modelo)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
tuned_model

In [ ]:
tuned_model = tune_model(modelo, n_iter=5)

In [ ]:
evaluate_model(tuned_model)

In [1]:
#Note:
#Precision: is the ratio of correctly predicted possitive observations to the total predicted positive observations
#Recall: is the ratio of correctly predicted possitive observations to the alll observations in actual class F1 score
#F1 score: is the weighted average of Precision and Recall